# Serverless cost curves

This notebook calculates costs for some EC2 and Lambda flavors by taking a uniform distribution of requests during a whole month


## 0. Initial setup

### Imports

In [ ]:
from bbva_colors import BBVAcolors
import bbva_lambda_tools as lambda_tools
import awscosts
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
import plotly.graph_objs as go
from plotly import tools

init_notebook_mode(connected=True)

### User variables

Modify this to play with different values

In [ ]:
ec2_flavors = ('m3.medium', 'm4.large', 'm4.4xlarge')
reqs_range = range(1, 3000)

lambda_memory = 128 # MiB
lambda_time = 200 # ms
throughput_ratio=1

image_type = None
# Uncomment next line to produce pngs
# image_type = 'png'

## 1. Generate and plot costs for uniform distribution of requests

### 1.1 Request generation and costs calculation

In [ ]:
cost = lambda_tools.generate_costs_in_month(
    requests_range=range(5, 3000),
    flavors=ec2_flavors,
    memory=lambda_memory,
    time=lambda_time,
)

### 1.2 Plot

In [ ]:
figure = lambda_tools.draw_costs_by_requests(cost)

iplot(figure, image=image_type)

## 2. Generate costs with a range of throughput ratios

In [ ]:
ratio_list = [1, 5, 10]

def generate_multi_ratio(ratio_list, requests_per_second, flavors, memory, time):
    costs = dict()
    for ratio in ratio_list:
        costs[ratio] = lambda_tools.generate_costs_in_month(
            reqs_range, ec2_flavors, lambda_memory, lambda_time, ratio
        )
    return costs

costs = generate_multi_ratio(ratio_list, reqs_range, ec2_flavors, lambda_memory, lambda_time)

In [ ]:
titles = []
rows_number = len(ratio_list)

row = 1

fig = tools.make_subplots(
    rows=rows_number,
    cols=1,
    shared_xaxes=False,
    subplot_titles = [
        f'Throughput ratio α={i} ({lambda_memory} MiB, {lambda_time} ms Lambda)' for i in ratio_list
    ]
)

for ratio, cost in costs.items():
    fig['layout']['xaxis{:d}'.format(row)].update(
        title='reqs/s',
        type='log',
    )
    fig['layout']['yaxis{:d}'.format(row)].update(
        title='Monthly cost ($)',
        type='log',
    )

    color = iter(
        [BBVAcolors['light'], BBVAcolors['aqua'], BBVAcolors['navy'], BBVAcolors['coral']]
    )
    showlegend = True if row == rows_number else False
    
    for flavor in cost.keys():
        x = list()
        y = list()
        for reqs, price in cost[flavor].items():
            x.append(reqs)
            y.append(price)

        trace = go.Scatter(
            x=x,
            y=y,
            name=flavor,
            marker=next(color),
            showlegend=showlegend,
        )
        fig.append_trace(trace, row, 1)
    row = row + 1

fig['layout'].update(
    title='<b>Monthly cost by number of requests per second</b>',
    legend=dict(
        orientation='h'
    ),
    width=950,
    height=400*rows_number,
)

iplot(
    fig,
    image=image_type,
    image_height=1024,
    image_width=1000,
)



In [ ]:
memory_list = [128, 1024, 2048]

param_list = memory_list

def generate_multi_memory(param_list, requests_per_second, flavors, time, throughput_ratio=1):
    costs = dict()
    for param in param_list:
        costs[param] = lambda_tools.generate_costs_in_month(
            reqs_range, ec2_flavors, param, lambda_time, throughput_ratio,
        )
    return costs

costs = generate_multi_memory(param_list, reqs_range, ec2_flavors, lambda_time)


In [ ]:
titles = []
rows_number = len(param_list)

row = 1

fig = tools.make_subplots(
    rows=rows_number,
    cols=1,
    shared_xaxes=False,
    subplot_titles = [
        f'Throughput ratio α={throughput_ratio} ({i} MiB, {lambda_time} ms Lambda)' for i in param_list
    ]
)


for param, cost in costs.items():
    fig['layout']['xaxis{:d}'.format(row)].update(
        title='reqs/s',
        type='log',
    )
    fig['layout']['yaxis{:d}'.format(row)].update(
        title='Monthly cost ($)',
        type='log',
    )

    color = iter(
        [BBVAcolors['light'], BBVAcolors['aqua'], BBVAcolors['navy'], BBVAcolors['coral']]
    )
    showlegend = True if row == rows_number else False
    
    for flavor in cost.keys():
        x = list()
        y = list()
        for reqs, price in cost[flavor].items():
            x.append(reqs)
            y.append(price)

        trace = go.Scatter(
            x=x,
            y=y,
            name=flavor,
            marker=next(color),
            showlegend=showlegend,
        )
        fig.append_trace(trace, row, 1)
    row = row + 1

fig['layout'].update(
    title='<b>Monthly cost by number of requests per second</b>',
    legend=dict(
        orientation='h'
    ),
    width=800,
    height=1000,
)

iplot(
    fig,
    image=image_type,
    image_height=1024,
    image_width=800,
)
